In [267]:
#import libraries

import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from IPython import display
import seaborn as sns
import os
import pandas as pd

#display.set_matplotlib_formats('svg')

#import data
data = pd.read_csv('d2l-en/Local/kaggleTitanic/resources/train.csv')
test_data = pd.read_csv('d2l-en/Local/kaggleTitanic/resources/test.csv')


# training_data['train_test'] = 1
# test_data['train_test'] = 0
# test_data['Survived'] = 0

# all_data = pd.concat([training_data, test_data])
#all_data.columns

# training_data.info()
# training_data.describe()

# df_num = training_data[['Age', 'SibSp', 'Parch', 'Fare']]
# df_cat = training_data[['Survived', 'Pclass', 'Sex', 'Ticket', 'Cabin', 'Embarked']]

# for i in df_num.columns:
#     plt.hist(df_num[i])
#     plt.title(i)
#     plt.show()


# print(df_num.corr()) #check correlation between numerical features
# sns.heatmap(df_num.corr())

# pd.pivot_table(training_data, index='Survived', values=['Age', 'SibSp', 'Parch', 'Fare'])


#check data shape
# data.info()

# data['train_test'] = 1

# for i in df_cat.columns:
#     sns.barplot(x=df_cat[i].value_counts().index, y=df_cat[i].value_counts()).set_title(i)
#     plt.show()

# print(pd.pivot_table(training_data, index='Survived', columns='Pclass', values='Ticket', aggfunc='count'))
# print(pd.pivot_table(training_data, index='Survived', columns='Sex', values='Ticket', aggfunc='count'))
# print(pd.pivot_table(training_data, index='Survived', columns='Embarked', values='Ticket', aggfunc='count'))

# training_data.head()

# clean data - fill missing values with mean, drop columns that are not useful, convert categorical data to numerical
def cleanData(data):
    data['Age'].fillna(data['Age'].mean(), inplace=True)
    data['Fare'].fillna(data['Fare'].mean(), inplace=True)
    data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)
    data.drop(['Cabin', 'Ticket', 'Name', 'PassengerId'], axis=1, inplace=True)

    embarked_mapping = {'S': 1, 'C': 2, 'Q': 3}
    sex_mapping = {'female': 1, 'male': 0}

    data['Embarked'] = data['Embarked'].map(embarked_mapping)
    data['Sex'] = data['Sex'].map(sex_mapping)

def buildModel(lr):
    model = nn.Sequential(
        nn.Linear(7, 64),
        nn.ReLU(),
        nn.Linear(64, 64),
        nn.ReLU(),
        nn.Linear(64, 2)
        # nie dodajemy softmaxa, bo jest juz w CrossEntropyLoss()
    )
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    return model, loss_fn, optimizer


def trainMode(epochs, data, labels, model, loss_fn, optimizer):

    losses = torch.zeros(epochs)
    ongoingAcc = []

    for epoch in range(epochs):

        #forward pass
        y_pred = model(data)

        #compute loss
        loss = loss_fn(y_pred, labels)
        losses[epoch] = loss

        #back propagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        #compute accuracy
        matches = torch.argmax(y_pred, axis=1) == labels #check if y_pred is equal to labels
        matchesNumeric = matches.float() #convert to float
        accuracyPct = 100 * torch.mean(matchesNumeric) #calculate accuracy
        ongoingAcc.append(accuracyPct) #append accuracy to list

    return losses, loss, accuracyPct, ongoingAcc
# cleanData(training_data)
# training_data.head()

def createAccuracyLossesPlot(losses, ongoingAcc):
    
    fix, ax = plt.subplots(1, 2, figsize=(13, 4))

    ax[0].plot(losses.detach())
    ax[0].set_title('Losses')
    ax[0].set_xlabel('Epoch')
    ax[0].set_ylabel('Loss')

    ax[1].plot(ongoingAcc)
    ax[1].set_title('Accuracy')
    ax[1].set_xlabel('Epoch')
    ax[1].set_ylabel('Accuracy (%)')

    plt.show()

def roundValues(model, data):
    preditcions = model(data)
    sm = nn.Softmax(dim=1)
    preditcions = sm(preditcions)
    preditcions = torch.argmax(preditcions, axis=1)
    return preditcions

lr = 0.01
epochs = 1000



cleanData(data)
#cleanData(test_data)

# print(test_data.head())
# print(data.head())
# print(data)
# print(data[data.columns[1:]])

# covert data to tensor and split into data and labels
labels = torch.tensor(data[data.columns[0]].values, dtype=torch.long)
data = torch.tensor(data[data.columns[1:]].values, dtype=torch.float32)

model, loss_fn, optimizer = buildModel(lr)
losses, loss, accuracyPct, ongoingAcc = trainMode(epochs, data, labels, model, loss_fn, optimizer)

# print(data)

# data_tensor

In [268]:

passangerId = torch.tensor(test_data[test_data.columns[0]].values, dtype=torch.long)
passangerId
cleanData(test_data)#test_data = torch.tensor(test_data, dtype=torch.float32)
test_data = torch.tensor(test_data[test_data.columns[:]].values, dtype=torch.float32)


In [287]:
passangerId, test_data

# Reshape passangerId to match the dimensions of test_data for concatenation

# Concatenate along dimension 1
# concat = torch.cat((passangerId, test_data), dim=1)
# concat

# createAccuracyLossesPlot(losses, ongoingAcc)

# preditcions = roundValues(model, data)

# test_data = torch.tensor(test_data, dtype=torch.float32)

test_prediction = roundValues(model, test_data)
test_prediction
final = torch.stack((passangerId, test_prediction), dim=1)
final
final_df = pd.DataFrame(final.numpy(), columns=['PassengerId', 'Survived'])
final_df
final_df.to_csv('d2l-en/Local/kaggleTitanic/resources/submission.csv', index=False)